In [5]:
import yfinance as yf
import pandas as pd
import datetime

# Get the current date as the end date
end_date = datetime.datetime.today().strftime('%Y-%m-%d')

# Example list of S&P 500 tickers (full list can be obtained elsewhere)
sp500_tickers = ["AAPL", "NVDA", "MSFT", "GOOG", "GOOGL", "AMZN", "META", "AVGO", "LLY", "TSLA", 
                 "WMT", "JPM", "V", "XOM", "UNH", "ORCL", "MA", "HD", "PG", "COST", "JNJ", 
                 "NFLX", "ABBV", "BAC", "KO", "CRM", "CVX", "MRK", "TMUS", "AMD", "PEP", 
                 "ACN", "LIN", "TMO", "MCD", "CSCO", "ADBE", "WFC", "IBM", "GE", "ABT", 
                 "DHR", "AXP", "MS", "CAT", "NOW", "QCOM", "PM", "ISRG", "VZ"]

# Download data for all tickers at once
data = yf.download(sp500_tickers, start='2003-01-01', end=end_date, group_by='ticker')

# The data now has a MultiIndex column: first level is price type, second level is ticker
# To have a cleaner dataframe, we can stack the price types and unstack the tickers

# Now, 'close_data' is a dataframe with dates as index and tickers as columns
print(data.head())


[*********************100%***********************]  50 of 50 completed

Ticker        AMZN                                                   NFLX  \
Price         Open    High     Low   Close Adj Close     Volume      Open   
Date                                                                        
2003-01-02  0.9595  0.9840  0.9275  0.9785    0.9785  129680000  0.846429   
2003-01-03  0.9795  1.0265  0.9705  1.0260    1.0260  175140000  0.850000   
2003-01-06  1.0200  1.0585  1.0150  1.0350    1.0350  153962000  0.785714   
2003-01-07  1.0415  1.0875  1.0300  1.0775    1.0775  191364000  0.771429   
2003-01-08  1.0720  1.0800  1.0410  1.0510    1.0510  139630000  0.771429   

Ticker                                    ...      ISRG                      \
Price           High       Low     Close  ...       Low     Close Adj Close   
Date                                      ...                                 
2003-01-02  0.900000  0.810714  0.817857  ...  1.333333  1.355556  1.355556   
2003-01-03  0.850000  0.771429  0.772857  ...  1.344444  1.355556  

In [6]:
print(data.tail(1))

Ticker            AMZN                                                  \
Price             Open        High         Low       Close   Adj Close   
Date                                                                     
2024-11-27  206.979996  207.639999  205.050003  205.740005  205.740005   

Ticker                      NFLX                                      ...  \
Price         Volume        Open        High         Low       Close  ...   
Date                                                                  ...   
2024-11-27  28061600  872.059998  880.950012  858.909973  877.340027  ...   

Ticker           ISRG                                         TMO              \
Price             Low       Close   Adj Close  Volume        Open        High   
Date                                                                            
2024-11-27  538.22998  538.549988  538.549988  938400  515.409973  528.859985   

Ticker                                                  
Price       

In [7]:
# data.to_parquet('sp500_50stocks_data.parquet')

In [8]:
data.columns

MultiIndex([('AMZN',      'Open'),
            ('AMZN',      'High'),
            ('AMZN',       'Low'),
            ('AMZN',     'Close'),
            ('AMZN', 'Adj Close'),
            ('AMZN',    'Volume'),
            ('NFLX',      'Open'),
            ('NFLX',      'High'),
            ('NFLX',       'Low'),
            ('NFLX',     'Close'),
            ...
            ('ISRG',       'Low'),
            ('ISRG',     'Close'),
            ('ISRG', 'Adj Close'),
            ('ISRG',    'Volume'),
            ( 'TMO',      'Open'),
            ( 'TMO',      'High'),
            ( 'TMO',       'Low'),
            ( 'TMO',     'Close'),
            ( 'TMO', 'Adj Close'),
            ( 'TMO',    'Volume')],
           names=['Ticker', 'Price'], length=300)